In [ ]:
!pip install py2neo
!pip install python-igraph
!pip install textblob

In [1]:
#import pandas as pd
from py2neo import Graph, Node, Relationship

In [2]:
# Initialise the graph db
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"
graph = Graph(uri=uri, user=user, password=password)

In [5]:
# Testing graph connection
query = "MATCH (n) RETURN COUNT(n) AS numberOfNodes"
run_query(query)

In [14]:
# Create constraints on nodes
graph.run("CREATE CONSTRAINT ON (c:Customer) ASSERT c.id IS UNIQUE")
graph.run("CREATE CONSTRAINT ON (p:Product) ASSERT p.id IS UNIQUE")

In [19]:
# Create new customer nodes taking into account constraints
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "file:///santander_cust_products_clean_sample.csv" AS row
MERGE (c:Customer {id: row.cust_id})
ON CREATE SET c.first_contract_date = row.first_contract_date, c.household_gross_income = row.household_gross_income
'''
graph.run(query)

In [21]:
# Create new product nodes taking into account constraints
query = """
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "file:///santander_products.csv" AS row
MERGE (p:Product {id: row.prod_id})
ON CREATE SET p.product_type = row.product_type
"""
run_query(query)

In [ ]:
# Create the edges between customers and the products they have - option 1
query = """
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS
FROM "url" AS row
MATCH (c:Customer {id: row.cust_id})
MATCH (p:Product {product_type: row.product_type})
MERGE (c)-[:HAS_PRODUCT]->(p)
"""
run_query(query)

In [ ]:
# Create the edges between customers and the products they have
query = """
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM "url" AS row

WITH split(row.products, ',') AS products, row.cust_id AS cust_id
UNWIND products AS product_type

WITH DISTINCT cust_id, product_type
MATCH (c:Customer {id: cust_id})
MATCH (p:Product {product_type: product_type})
MERGE (p)-[:HAS_PRODUCT]->(product_type)
"""

run_query(query)